In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
import random
import tensorflow as tf
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, LeakyReLU,BatchNormalization, Add,Dropout
from tensorflow.keras.models import Model
from segmentation_models.metrics import IOUScore
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical


In [ ]:

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
train_img_dir = "data/data_for_keras_aug/train_images/train/"
train_mask_dir = "data/data_for_keras_aug/train_masks/train/"

In [ ]:
img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

In [ ]:
num_images = len(os.listdir(train_img_dir))

In [ ]:
img_num = random.randint(0, num_images-1)

In [ ]:
img_for_plot = cv2.imread(train_img_dir+img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)
mask_for_plot =cv2.imread(train_mask_dir+msk_list[img_num], 0)
plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()


## Defining seed size

In [ ]:
seed=24
batch_size= 8
n_classes=2

### BackBone

### Preprocessing

In [ ]:
#Use this to preprocess input for transfer learning
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
scaler = MinMaxScaler()
def preprocess_data(img, mask, num_class):
    #Scale images
    img = scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)
    img = preprocess_input(img)  #Preprocess based on the pretrained backbone...
    #Convert mask to one-hot
    mask = mask / 255.0  # Normalize mask values to [0, 1]
    mask = to_categorical(mask, num_class)
      
    return (img,mask)

## Train Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler  # Assuming StandardScaler is used

def trainGenerator(train_img_path, train_mask_path, num_class):
    img_data_gen_args = dict(
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=30,           # Random rotations
        width_shift_range=0.1,       # Random width shifts
        height_shift_range=0.1,      # Random height shifts
        zoom_range=0.2,              # Random zoom
        fill_mode='reflect'
    )
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)
    mask_datagen = ImageDataGenerator(**img_data_gen_args)
    
    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        target_size=(256, 256),  # Adjust based on your model input size
        color_mode='rgb',        # Assuming RGB images
        class_mode=None,
        batch_size=batch_size,
        seed=seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_mask_path,
        target_size=(256, 256),  # Adjust based on your model input size
        color_mode='grayscale',
        class_mode=None,
        batch_size=batch_size,
        seed=seed)
    
    train_generator = zip(image_generator, mask_generator)
    
    for (img, mask) in train_generator:
        img, mask = preprocess_data(img, mask, num_class)
        yield img, mask



In [ ]:
train_img_path = "data/data_for_keras_aug/train_images/"
train_mask_path = "data/data_for_keras_aug/train_masks/"
train_img_gen = trainGenerator(train_img_path, train_mask_path, num_class=2)

In [ ]:
x, y = train_img_gen.__next__()

In [ ]:
for i in range(0,3):
    image = x[i]
    mask = np.argmax(y[i], axis=2)
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.show()

In [ ]:
val_img_path = "data/data_for_keras_aug/val_images/"
val_mask_path = "data/data_for_keras_aug/val_masks/"
val_img_gen = trainGenerator(val_img_path, val_mask_path, num_class=2)

In [ ]:
x_val, y_val = val_img_gen.__next__()

In [ ]:
for i in range(0,2):
    image = x_val[i]
    mask = np.argmax(y_val[i], axis=2)
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.show()

In [ ]:
num_train_imgs = len(os.listdir('data/data_for_keras_aug/train_images/train/'))
num_val_images = len(os.listdir('data/data_for_keras_aug/val_images/val/'))
steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size

In [ ]:
IMG_HEIGHT = x.shape[1]
IMG_WIDTH  = x.shape[2]
IMG_CHANNELS = x.shape[3]


In [ ]:
IMG_CHANNELS

In [ ]:
y.shape[3]

In [ ]:
n_classes=2

### Custom Loss

In [ ]:
import tensorflow as tf


class DiceLoss(tf.keras.losses.Loss):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def call(self, y_true, y_pred):
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
        dice = (2.0 * intersection + 1e-7) / (union + 1e-7)
        return 1.0 - dice  


def custom_loss(y_true, y_pred):
    y_true_float = tf.cast(y_true, tf.float32)
    dice_loss = DiceLoss()(y_true_float, y_pred)
    l2_reg = 0.01  
    l2_loss = l2_reg * tf.reduce_sum([tf.reduce_mean(tf.square(w)) for w in tf.compat.v1.trainable_variables()])
    focal_loss_value = sm.losses.categorical_focal_loss(y_true, y_pred)
    combined_loss = dice_loss + l2_loss
    
    return combined_loss


### Total Loss

In [ ]:

dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

### Model

In [ ]:
from segmentation_models.losses import DiceLoss
from segmentation_models.metrics import IOUScore
from tensorflow.keras.optimizers import Adam
model = sm.Unet(BACKBONE, encoder_weights='imagenet', 
                input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                classes=2, activation='softmax')

##Try playing around with different loss functions
model.compile(optimizer=Adam(1e-4), loss= DiceLoss(),  metrics=['accuracy', IOUScore()])



### FCN

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50  # Substitute with ResNet34 
from tensorflow.keras.optimizers import Adam

### This is an implementation of FCN (Fully Convolutional Networks) 
### You can see slight changes in the architectures of this and the Unet in the next code block.

def build_fcn_32s(input_shape, num_classes):
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in resnet.layers:
        layer.trainable = False  # Freeze ResNet layers

    # Get the output of the ResNet encoder (last pooling layer output)
    encoder_output = resnet.output  # Shape should be (8, 8, 2048) for ResNet50

    # Convolution layer for class predictions
    x = Conv2D(512, (7, 7), activation='relu', padding='same')(encoder_output)
    x = Conv2D(512, (1, 1), activation='relu', padding='same')(x)
    x = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    upsampled_output = UpSampling2D(size=(32, 32), interpolation='bilinear')(x)

    
    model = Model(inputs=resnet.input, outputs=upsampled_output)
    return model

# Set parameters
input_shape = (256, 256, 3)  
num_classes = 2 

# Build and compile the model
model = build_fcn_32s(input_shape, num_classes)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


### Custom UNet

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout
from tensorflow.keras.models import Model

def multi_unet_model(n_classes=2, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3):
    # Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    # Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    ### This is a learning rate scheduler, it decays the learning rate as the model learns, we can set the number of steps after which the learning rate should start decaying and the factor by which it decays.
    ### Change the decay steps based on the number of epochs. May be you can try number of epochs/10 as the decay steps, so for 25 may be give 2 steps per decay and try increasing fromm there.
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=2,
    decay_rate=0.9)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    ##try to play with loss functions,hyper parameters like number of filters,optimizer learning rates.
    model.compile(optimizer=Adam(), loss=sm.losses.cce_jaccard_loss, metrics=['accuracy', IOUScore()])
    ### model.summary gives you the idea of the components of the model and its architecture. You can get a better understanding of the models config.
    model.summary()
    return model


model = multi_unet_model(n_classes=2, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3)


### Unet + LSTM

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, LSTM, Reshape
from tensorflow.keras.models import Model
from segmentation_models.metrics import IOUScore
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def conv_block(inputs, filters, kernel_size=3, activation='relu', padding='same'):
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
    return conv

def build_unet_with_lstm(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = conv_block(pool4, 1024)
    
    # Reshape before LSTM
    lstm_input_shape = (16*16, 1024)  # Flattened to treat as a sequence
    lstm = Reshape(lstm_input_shape)(conv5)
    
    # LSTM layer
    lstm = LSTM(512, return_sequences=True)(lstm)
    
    # Reshape after LSTM
    lstm_output_shape = (16, 16, 512)  # Reshape back to original shape
    lstm = Reshape(lstm_output_shape)(lstm)
    
    # Decoder
    up6 = concatenate([UpSampling2D(size=(2, 2))(lstm), conv4], axis=-1)
    conv6 = conv_block(up6, 512)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = conv_block(up7, 256)
    
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = conv_block(up8, 128)
    
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = conv_block(up9, 64)
    
    # Output layer
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define hyperparameters
lr = 3e-4
batch_size = 16
epochs = 50


model_with_lstm = build_unet_with_lstm(input_shape=(256, 256, 3), num_classes=2)
model_with_lstm.compile(optimizer=Adam(), loss=DiceLoss(), metrics=['accuracy', IOUScore()])

model_with_lstm.summary()

### Unet +LSTM and Dropout

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, LSTM, Reshape, Dropout
from tensorflow.keras.models import Model
from segmentation_models.metrics import IOUScore
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from segmentation_models.losses import DiceLoss
from segmentation_models.metrics import FScore

def conv_block(inputs, filters, kernel_size=3, activation='relu', padding='same'):
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
    conv = Dropout(0.2)(conv)  # Adding dropout layer with a dropout rate of 20%
    return conv

def build_unet_with_lstm(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = conv_block(pool4, 1024)
    
    # Reshape before LSTM
    lstm_input_shape = (16*16, 1024)  # Flattened to treat as a sequence
    lstm = Reshape(lstm_input_shape)(conv5)
    
    # LSTM layer
    lstm = LSTM(512, return_sequences=True)(lstm)
    
    # Reshape after LSTM
    lstm_output_shape = (16, 16, 512)  # Reshape back to original shape
    lstm = Reshape(lstm_output_shape)(lstm)
    
    # Decoder
    up6 = concatenate([UpSampling2D(size=(2, 2))(lstm), conv4], axis=-1)
    conv6 = conv_block(up6, 512)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = conv_block(up7, 256)
    
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = conv_block(up8, 128)
    
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = conv_block(up9, 64)
    
    # Output layer
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define hyperparameters
learning_rate = 3e-4
batch_size = 16
epochs = 50
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=3e-4,
    decay_steps=200,
    decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate)
model_with_lstm = build_unet_with_lstm(input_shape=(256, 256, 3), num_classes=2)
dice_metric = FScore(name='dice_coefficient', beta=1)

model_with_lstm.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy",   metrics=['accuracy', IOUScore(), dice_metric])

model_with_lstm.summary()

### Unet + GRU

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, GRU, Reshape, Dropout, LayerNormalization,Conv2DTranspose
from tensorflow.keras.models import Model
from segmentation_models.metrics import IOUScore
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from segmentation_models.losses import DiceLoss
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import HeNormal


def conv_block(inputs, filters, kernel_size=3, activation='leaky_relu', padding='same'):
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
    
    conv = Dropout(0.2)(conv)  # Adding dropout layer with a dropout rate of 20%
    return conv


def build_unet_with_gru(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = conv_block(pool4, 1024)
    
    # Reshape before GRU
    gru_input_shape = (16*16, 1024)  # Flattened to treat as a sequence
    gru = Reshape(gru_input_shape)(conv5)
    
    # GRU layer
    gru = GRU(512, return_sequences=True)(gru)
    
    # Reshape after GRU
    gru_output_shape = (16, 16, 512)  # Reshape back to original shape
    gru = Reshape(gru_output_shape)(gru)
    
    # Decoder
    up6 = concatenate([UpSampling2D(size=(2, 2))(gru), conv4], axis=-1)
    conv6 = conv_block(up6, 512)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = conv_block(up7, 256)
    
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = conv_block(up8, 128)
    
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = conv_block(up9, 64)
    
    # Output layer
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


# Define hyperparameters
learning_rate = 3e-4
batch_size = 16
epochs = 50
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10,
    decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(1e-4)
model_with_gru = build_unet_with_gru(input_shape=(256, 256, 3), num_classes=2)

model_with_gru.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=['accuracy', IOUScore()])

model_with_gru.summary()


### Train

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint

# Checkpoint to save the best model based on val_loss
checkpoint = ModelCheckpoint('test.hdf5',
                             monitor='val_loss',  # Track validation loss
                             save_best_only=True,  
                             mode='min',  # Minimize validation loss
                             verbose=1)
### try changing the epochs
callbacks = [checkpoint]
history=model.fit(train_img_gen,
          steps_per_epoch=steps_per_epoch,
          epochs=25,
          verbose=1,
          validation_data=val_img_gen,
          validation_steps=val_steps_per_epoch,
          callbacks=callbacks
         )

### Loss Curve

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.show()

### IOU Curve

In [ ]:
acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.show()

### Accuracy Curve

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'y', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


### Model Evaluation

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
### give your model's name here
model = load_model("model.hdf5", compile=False)


test_image_batch, test_mask_batch = val_img_gen.__next__()


test_mask_batch_argmax = np.argmax(test_mask_batch, axis=3)
test_pred_batch = model.predict(test_image_batch)
test_pred_batch_argmax = np.argmax(test_pred_batch, axis=3)

# Calculate accuracy
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(test_mask_batch_argmax, test_pred_batch_argmax)
print("Accuracy =", accuracy.result().numpy())

n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(test_pred_batch_argmax, test_mask_batch_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

# Flatten the arrays for classification report and confusion matrix
test_mask_flat = test_mask_batch_argmax.flatten()
test_pred_flat = test_pred_batch_argmax.flatten()

# Calculate and print the classification report
class_report = classification_report(test_mask_flat, test_pred_flat)
print("Classification Report:\n", class_report)

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(test_mask_flat, test_pred_flat)
print("Confusion Matrix:\n", conf_matrix)



In [ ]:
for x in range(0, test_image_batch.shape[0]-1):
    img_num=x
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_image_batch[img_num])
    
    plt.subplot(232)
    plt.title('Testing Label')

    plt.imshow(test_mask_batch_argmax[img_num],cmap='gray')
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(test_pred_batch_argmax[img_num],cmap='magma')
    plt.show()